In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import transformers

print(transformers.__version__)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:

dataset_path = '/content/drive/MyDrive/T5/dataset.csv'
df = pd.read_csv(dataset_path)
df.head(1)

,text,summary
0,long term synaptic plasticity thought represen...,understanding spatiotemporal organization long...


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

In [ ]:
# Drop rows with NaN values in 'text' or 'summary' columns
train_df = train_df.dropna(subset=['text', 'summary'])
test_df = test_df.dropna(subset=['text', 'summary'])

In [ ]:
pip install datasets

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
max_input_length = 512
max_target_length = 128

In [ ]:
def preprocess_batch(batch):
    inputs = ["summarize: " + text for text in batch["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    labels = tokenizer(batch["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(preprocess_batch, batched=True)
test_dataset = test_dataset.map(preprocess_batch, batched=True)


Map:   0%|          | 0/855 [00:00<?, ? examples/s]

Map:   0%|          | 0/94 [00:00<?, ? examples/s]

In [ ]:
train_dataset = train_dataset.remove_columns(["text", "summary"])
test_dataset = test_dataset.remove_columns(["text", "summary"])


In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/T5",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=30,
    predict_with_generate=True,
    fp16=True,
    logging_dir="/content/drive/MyDrive/T5",
    logging_steps=100,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import EarlyStoppingCallback, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

early_stopping = EarlyStoppingCallback(early_stopping_patience=3)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[early_stopping]
)

<ipython-input-32-44c26bdc56fe>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.877500,3.878467
2,3.837900,3.846636
3,3.861900,3.827939
4,3.805200,3.806808
5,3.828400,3.789482
6,3.798400,3.778465
7,3.718000,3.765348
8,3.649500,3.757657
9,3.677300,3.755388
10,3.651700,3.742968


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=6420, training_loss=3.6220064573198836, metrics={'train_runtime': 1091.4766, 'train_samples_per_second': 23.5, 'train_steps_per_second': 5.882, 'total_flos': 3471517207756800.0, 'train_loss': 3.6220064573198836, 'epoch': 30.0})

In [ ]:
model.save_pretrained("/content/drive/MyDrive/T5/t5small_model")
tokenizer.save_pretrained("/content/drive/MyDrive/T5/t5small_tooken")


('/content/drive/MyDrive/T5/t5small_tooken/tokenizer_config.json',
 '/content/drive/MyDrive/T5/t5small_tooken/special_tokens_map.json',
 '/content/drive/MyDrive/T5/t5small_tooken/spiece.model',
 '/content/drive/MyDrive/T5/t5small_tooken/added_tokens.json',
 '/content/drive/MyDrive/T5/t5small_tooken/tokenizer.json')